<a href="https://colab.research.google.com/github/hunter-z-hunter/hunter-z-hunter/blob/main/hunter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Hunter Z Hunter! 

[Hunter z Hunter](https://github.com/hunter-z-hunter) is a scavenger hunt game that anyone can use to automatically reward successful hunters with ether. We use a lightweight machine learning model to determine if the image submitted matches the target image of the treasure. We use [ezkl](https://github.com/zkonduit/ezkl) to generate a zero knowledge circuit of this model so that it can remain private and run on-chain for automatic crypto payments.

Below is our lightweight neural net.

Here we use Pytorch and Numpy to define a neural network called "Hunt." Hunt is used to 

In [ ]:
import torch
import torch.nn as nn
import numpy as np

class Hunt(nn.Module):
    def __init__(self):
        super(Hunt, self).__init__()
        # self.weight = nn.Parameter(torch.randn(28 * 28))
        # linear, relu, linear multilayer perception
        # conv2d relu, [(conv2d, relu), ...], linear
        
    def forward(self, x, target):
        x = x.view(-1, 28*28)
        target = target.view(-1, 28*28)
        return torch.sqrt(torch.sum((x - target) ** 2, dim=1))


In [ ]:
!pip install ezkl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ezkl

This is a more complex 

In [ ]:
from torch import nn
from ezkl import export

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)
        
        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)
    #    x = x.flatten()

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)
       
        return logits

circuit = MyModel()
export(circuit, input_shape = [1,28,28])


    

3L Relu

In [ ]:
from torch import nn
from ezkl import export

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1,4, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(4,4, kernel_size=5, stride=2)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(4*4*4, 10)

    def forward(self, x):
        x = x.view(-1,1,28,28)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1,4*4*4)
        x = self.fc(x)
        return x

circuit3L = MyModel()
export(circuit3L, input_shape = [1,28,28])

We then create an instance of our model, pass in some random data (soon to be image vectors), and determine the Euclidian distance between them.

In [ ]:
model = Hunt()
input_image = torch.randn(1, 1, 28, 28)
target_image = torch.randn(1, 1, 28, 28)
distance = model(input_image, target_image)
print(distance)

tensor([37.8969])


In [ ]:
import torch

input_image = torch.randn(1, 1, 28, 28)
fourrelu = circuit(input_image)
print(fourrelu)

tensor([[-0.0016,  0.1354, -0.0430, -0.1704,  0.0173,  0.0581, -0.1116,  0.0928,
          0.1473, -0.0139]], grad_fn=<AddmmBackward0>)


In [ ]:
import json

In [ ]:
import torch

After importing the JSON python file, we use the export function from ezkl to create a network.onnx file and an input.json file. These files are what ezkl inputs to generate a verifier for a model. 

In [ ]:
def export():
    torch_model = model
    # Input to the model
    preimageShape = [3]
    targetShape = [3]
    x = 0.1*torch.rand(1,*preimageShape, requires_grad=True)
    y = 0.1*torch.rand(1,*targetShape, requires_grad=True)
    torch_out = torch_model(x, y)
    # Export the model
    torch.onnx.export(torch_model,               # model being run
                      (x,y),                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

    d = ((x).detach().numpy()).reshape([-1]).tolist()
    dy = ((y).detach().numpy()).reshape([-1]).tolist()


    data = dict(input_shapes = [preimageShape, targetShape],
                input_data = [d, dy],
                output_data = [((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

    # Serialize data into file:
    json.dump( data, open( "input.json", 'w' ) )

In [ ]:
def export4():
    torch_model = circuit
    # Input to the model
    preimageShape = [1, 28, 28]
    x = 0.1*torch.rand(1,*preimageShape, requires_grad=True)
    torch_out = torch_model(x)
    # Export the model
    torch.onnx.export(torch_model,               # model being run
                      (x),                   # model input (or a tuple for multiple inputs)
                      "network4.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

    d = ((x).detach().numpy()).reshape([-1]).tolist()


    data = dict(input_shapes = [preimageShape],
                input_data = [d],
                output_data = [((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

    # Serialize data into file:
    json.dump( data, open( "input4.json", 'w' ) )

In [ ]:
def export3L():
    torch_model = circuit3L
    # Input to the model
    preimageShape = [1, 28, 28]
    x = 0.1*torch.rand(1,*preimageShape, requires_grad=True)
    torch_out = torch_model(x)
    # Export the model
    torch.onnx.export(torch_model,               # model being run
                      (x),                   # model input (or a tuple for multiple inputs)
                      "network3.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

    d = ((x).detach().numpy()).reshape([-1]).tolist()


    data = dict(input_shapes = [preimageShape],
                input_data = [d],
                output_data = [((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

    # Serialize data into file:
    json.dump( data, open( "input3.json", 'w' ) )

We call export and will be using the ONNX file for our project!

In [ ]:
export()

TypeError: ignored

In [ ]:
export4()

In [ ]:
export3L()